In [31]:
%config Application.log_level="WORKAROUND"
%config Application.log_level="DEBUG"

import logging
logging.getLogger("notebook").setLevel(logging.INFO)

# Set to DEBUG to see what's going on under the hood
logging.getLogger("sqltables").setLevel(logging.INFO)

ERROR:root:The 'log_level' trait of an IPKernelApp instance must be any of (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL'), but a value of 'WORKAROUND' <class 'str'> was specified.


In [33]:
# Example from the README
db = sqltables.Database()
tab = db.load_values([[1], [2], [3]], column_names=["val"])

def square(tab):
    return tab.view("select val, val*val as squared from _")

square(tab)

|val|squared|
|-|-|
|1|1|
|2|4|
|3|9|


# Loading, querying and displaying tables

In [19]:
import sqltables

# Create a new in-memory database
db = sqltables.Database()

# Load values 0...9 from an iterable of rows
tab = db.load_values([[i] for i in range(10)], column_names="a")
tab

|a|
|-|
|0|
|1|
|2|
|3|
|4|
|5|
|6|
|7|
|8|
|9|


In [20]:
# Create a view with the values squared
squared = tab.view("select a, a*a as a2 from _")
squared

|a|a2|
|-|-|
|0|0|
|1|1|
|2|4|
|3|9|
|4|16|
|5|25|
|6|36|
|7|49|
|8|64|
|9|81|


In [21]:
# Access values through the iterator interface
it = iter(squared)
it.column_names, list(it)

(['a', 'a2'],
 [Row(a=0, a2=0),
  Row(a=1, a2=1),
  Row(a=2, a2=4),
  Row(a=3, a2=9),
  Row(a=4, a2=16),
  Row(a=5, a2=25),
  Row(a=6, a2=36),
  Row(a=7, a2=49),
  Row(a=8, a2=64),
  Row(a=9, a2=81)])

# Numpy interface

In [13]:
import numpy as np

In [14]:
np.array(list(squared))

array([[ 0,  0],
       [ 1,  1],
       [ 2,  4],
       [ 3,  9],
       [ 4, 16],
       [ 5, 25],
       [ 6, 36],
       [ 7, 49],
       [ 8, 64],
       [ 9, 81]])

# Query parameters

In [17]:
tab = squared.table("select a from _ where a >= ?", [2])
for [n] in tab:
    [[sum]] = squared.table("select sum(a2) from _ where a <= :n", {"n": n})
    print(f"{n}: {sum}")

2: 5
3: 14
4: 30
5: 55
6: 91
7: 140
8: 204
9: 285


# Joins

In [24]:
# Create a table with 5 random values between 0 and 7

import random

rand_rows = [[random.randint(0, 8)] for _ in range(5)]
rand_tab = db.load_values(rand_rows, column_names=["u"])
rand_tab

|u|
|-|
|6|
|4|
|1|
|3|
|8|


In [23]:
# Specify additional tables to join with in the bindings parameter
squared.view("select * from _, rand where a < u order by u", 
             bindings={"rand": rand_tab})

|a|a2|u|
|-|-|-|
|0|0|2|
|1|1|2|
|0|0|5|
|0|0|5|
|1|1|5|
|1|1|5|
|2|4|5|
|2|4|5|
|3|9|5|
|3|9|5|
|4|16|5|
|4|16|5|
|0|0|7|
|1|1|7|
|2|4|7|
|3|9|7|
|...|...|...|
